In [1]:
import pandas as pd
import torch
import data_process as dp
import embed
import matplotlib.pyplot as plt
import umap

device = "cuda" if torch.cuda.is_available() else "cpu"

2024-03-04 04:29:58.843291: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-04 04:29:58.887741: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-04 04:29:58.887775: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-04 04:29:58.888906: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-04 04:29:58.895693: I tensorflow/core/platform/cpu_feature_guar

In [2]:
model = embed.load_model("./checkpoint-large-3.pt", device)

In [3]:
model.to(device)
model.eval()
pass

In [4]:
df = pd.read_parquet("./datasets/pdb-no-model-no-asm.parquet")
dfv = pd.read_parquet("./datasets/all-venome.parquet")

In [5]:
dfv["name"] = dfv["name"].apply(lambda x: "VENOM_"+x)

In [6]:
df = pd.concat((df[["name", "3Di"]], dfv[["name", "3Di"]]))

In [7]:
df

,name,3Di
0,pdb101m.ent.gz,DADDVVLLVLQVVLVVVCVVPQLVLLLQLVLQLCVVPVVVLVLDPV...
1,pdb102l.ent.gz,DALVVLLCVVQPAAFFFDQDLVGFTAHGRGHGQDPDPDDVPRRQVS...
2,pdb102m.ent.gz,DADDVVLLCLQVVLVVVCVVPQLVLLLQLVLLLCVVPVVVLVLPPV...
3,pdb103l.ent.gz,DALLVLQCVVFPAAFFWDQDLVGFTDHGRRRGQDPPVSSQVSVCVL...
4,pdb103m.ent.gz,DADDVVLLVLQVVLVVVCVVPQLVLLLQLVLLLCVVPVVVQVLDPP...
...,...,...
431,VENOM_Lh14_comp82_c1_seq3.pdb,DVPPLVPPDPPDDDDDDQPADDPVLLVLLVVLLVCLLVVLCVQCVP...
432,VENOM_Lh14_comp835_c0_seq1.pdb,DDFFDFDDQDWFQWLVGIDGCLVVLAQAKEKEWEAFDFPDDLVLVQ...
433,VENOM_Lh14_comp862_c0_seq1.pdb,DPPPVVVVVVVVVVVVVVLVVVQVVVVVAPPPDDDDDDDDDDDDDD...
434,VENOM_Lh14_comp873_c0_seq1.pdb,DDDDDDDDDDDDDDDPPPDLLVCLVVQDDPVLNVLLVVLCCQLVDP...


In [10]:
res = embed.to_avg_embeddings_fast(df.iloc[:20], model, 256, device)

/home/bertuccd/protein-map/experiments/embed.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["len"] = df["3Di"].apply(lambda x: len(x))
100%|██████████| 8/8 [00:00<00:00, 54.48it/s]


In [11]:
res

([tensor([ -8.8609,  -4.1198,  -3.8645, -10.3362,  -2.9292,   7.2168,   2.6455,
            8.3548,   7.7392,   2.0421,   8.5692, -14.9461,   9.4960,  -3.9926,
            1.9258,  -3.3060,  -8.0244,  -2.1792,   9.5317,   9.1032,  -8.2536,
            1.1109,   3.9841,  -6.8236, -11.8143,   3.6167,  -8.3361,  -4.1825,
          -11.4748,   6.8062,  -4.3012,  12.6931,   5.4044,  -7.7825,  19.7487,
            9.2205,  14.3086,   4.1725,  -9.4789,   1.2532,  -3.2860, -11.9251,
            8.9014,  -3.9497,  -6.1108,   1.5955, -15.4132,  -6.2056,  12.1333,
           11.9033,   7.7762,   2.1015,  -9.4006,  -1.3077, -15.8201,  14.4666,
            8.6417,   0.2806,  -4.0470,   4.4258, -10.1247, -13.4961,  19.5488,
            5.8819,   2.0588,  13.7463,  -6.2730,   5.5414,   2.5722,  -4.2594,
            6.7633,   6.3635,   2.2860,  -7.3375,   9.8217,  -3.3714,  -0.8512,
           10.0956,  -9.6058,  -1.5925, -12.0270,  17.2439,   9.1022,  -8.0882,
            4.4991, -14.1997,   3.3401, 

In [12]:
res = embed.to_avg_embeddings(df.iloc[:20], model, 256, device)

100%|██████████| 20/20 [00:00<00:00, 67.31it/s]


In [13]:
res

tensor([[-13.1401,   1.1296, -11.0928,  ...,  -3.0027,  -7.9582,   4.2890],
        [ -3.3235,  -0.4013,  -2.0087,  ...,   7.8940, -10.9630,  -2.7587],
        [-13.1508,   0.0671, -13.4161,  ...,  -4.2852, -11.2084,   3.2192],
        ...,
        [ -5.8295, -15.2853, -12.5470,  ...,   5.0283, -13.2425,   4.5644],
        [-11.3352,   0.5136, -15.4698,  ...,  -4.1582,  -9.2875,   5.3090],
        [-10.0110,  -9.1160, -13.3733,  ...,   1.3286, -10.8848,   8.1871]])